In [1]:
!wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
!unzip song_lyrics.zip -d .

--2021-02-01 02:55:21--  https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
Resolving aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)... 52.239.148.4
Connecting to aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)|52.239.148.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2101791 (2.0M) [application/zip]
Saving to: ‘song_lyrics.zip’

song_lyrics.zip     100%[===================>]   2.00M  1.37MB/s    in 1.5s    

2021-02-01 02:55:23 (1.37 MB/s) - ‘song_lyrics.zip’ saved [2101791/2101791]

Archive:  song_lyrics.zip
  inflating: ./Kanye_West.txt        
  inflating: ./Lil_Wayne.txt         
  inflating: ./adele.txt             
  inflating: ./al-green.txt          
  inflating: ./alicia-keys.txt       
  inflating: ./amy-winehouse.txt     
  inflating: ./beatles.txt           
  inflating: ./bieber.txt            
  inflating: ./bjork.txt             
  inflating: ./blink-182.

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해)
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
import glob
from sklearn.model_selection import train_test_split

txt_file_path = '/content/*.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['What do you mean?', 'Oh, oh, oh', 'When you sometimes say yes']


In [3]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다.
        test_tensor = tf.concat([test_tensor,
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제

    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()
    
    # 토큰의 개수가 15보다 큰 문장 제외
    if len(sentence.split()) > 15:
        splitted_str = sentence.split()
        temp=''
        for idx, s in enumerate(splitted_str):
            if idx > 15:
                break
            temp.join(s)
        sentence = temp

        # sentence = str(sentence.split()[:15])


    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다

    return sentence


def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    print(tensor,tokenizer)
    return tensor, tokenizer


In [4]:
# val-loss를 목표치로 맞추기위해 이 부분만 따로 실행하도록 배치
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out


In [6]:

corpus = []

# 이 프로젝트에서는 길이가 0인 즉 빈줄만 제
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    corpus.append(preprocess_sentence(sentence))

print("데이터 크기:", len(corpus))
print("Examples:\n", corpus[:3])

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)

# 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

model.summary()

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)


데이터 크기: 175986
Examples:
 ['<start> what do you mean ? <end>', '<start> oh , oh , oh <end>', '<start> when you sometimes say yes <end>']
[[  2  39  48 ...   0   0   0]
 [  2  45   5 ...   0   0   0]
 [  2  47   7 ...   0   0   0]
 ...
 [  2  41 894 ...   0   0   0]
 [  2  41  68 ...   0   0   0]
 [  2   8  83 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f18716bc7b8>
Source Train: (140788, 16)
Target Train: (140788, 16)
Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
d

In [7]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
4400/4400 [==============================] - 248s 56ms/step - loss: 2.8651 - val_loss: 2.3928
Epoch 2/10
4400/4400 [==============================] - 247s 56ms/step - loss: 2.2819 - val_loss: 2.2193
Epoch 3/10
4400/4400 [==============================] - 246s 56ms/step - loss: 2.0229 - val_loss: 2.1224
Epoch 4/10
4400/4400 [==============================] - 240s 54ms/step - loss: 1.7875 - val_loss: 2.0625
Epoch 5/10
4400/4400 [==============================] - 236s 54ms/step - loss: 1.5877 - val_loss: 2.0235
Epoch 6/10
4400/4400 [==============================] - 236s 54ms/step - loss: 1.4161 - val_loss: 2.0112
Epoch 7/10
4400/4400 [==============================] - 235s 53ms/step - loss: 1.2814 - val_loss: 2.0113
Epoch 8/10
4400/4400 [==============================] - 236s 54ms/step - loss: 1.1648 - val_loss: 2.0204
Epoch 9/10
4400/4400 [==============================] - 235s 53ms/step - loss: 1.0752 - val_loss: 2.0461
Epoch 10/10
4400/4400 [==============================] 

In [8]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you <end> '

### 별다른 모델 수정 없이 val-loss가 2.0771로 10 epoch만에 낮아졌다.
### 물론 현재 training shape를 보면 프로젝트 지시사항에 나오는 데이터보다 많이 존재한다.
### 프로젝트 지시사항에서는 전처리후 나온 데이터셋 크기가 124960 인데 여기서는 140788이 나온다.